In [10]:
import os
import pandas as pd
import openpyxl as xl
from pathlib import Path

In [11]:
# Define the directory path
crosswalk_dir = Path("../data/raw/Crosswalks/Crosswalks")

# Get all .xlsx files in the folder
xlsx_files = sorted(crosswalk_dir.glob("CW*.xlsx"))

In [12]:
# Preview list of files
print("\nLoaded files:")
for f in xlsx_files:
    print(" -", f.name)


Loaded files:
 - CW2000.xlsx
 - CW2001.xlsx
 - CW2002.xlsx
 - CW2003.xlsx
 - CW2004.xlsx
 - CW2005.xlsx
 - CW2006.xlsx
 - CW2007.xlsx
 - CW2008.xlsx
 - CW2009.xlsx
 - CW2010.xlsx
 - CW2011.xlsx
 - CW2012.xlsx
 - CW2013.xlsx
 - CW2014.xlsx
 - CW2015.xlsx
 - CW2016.xlsx
 - CW2017.xlsx
 - CW2018.xlsx
 - CW2019.xlsx
 - CW2020.xlsx
 - CW2021.xlsx
 - CW2022_preliminary.xlsx


In [13]:
# Dictionary to hold all crosswalk dataframes
crosswalk_data = {}

pd.read_excel

<function pandas.io.excel._base.read_excel(io, sheet_name: 'str | int | list[IntStrT] | None' = 0, *, header: 'int | Sequence[int] | None' = 0, names: 'SequenceNotStr[Hashable] | range | None' = None, index_col: 'int | str | Sequence[int] | None' = None, usecols: 'int | str | Sequence[int] | Sequence[str] | Callable[[str], bool] | None' = None, dtype: 'DtypeArg | None' = None, engine: "Literal['xlrd', 'openpyxl', 'odf', 'pyxlsb', 'calamine'] | None" = None, converters: 'dict[str, Callable] | dict[int, Callable] | None' = None, true_values: 'Iterable[Hashable] | None' = None, false_values: 'Iterable[Hashable] | None' = None, skiprows: 'Sequence[int] | int | Callable[[int], object] | None' = None, nrows: 'int | None' = None, na_values=None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, parse_dates: 'list | dict | bool' = False, date_parser: 'Callable | lib.NoDefault' = <no_default>, date_format: 'dict[Hashable, str] | str | None' = None, thousands: 's

In [14]:
# Output results to a text file
output_file = Path("../data/processed/crosswalk_preview_output.txt")
with open(output_file, "w", encoding="utf-8") as out:
    for file in xlsx_files:
        try:
            xl = pd.ExcelFile(file)
            sheet_names = xl.sheet_names
            out.write(f"\n{file.name} contains sheets: {sheet_names}\n")

            # Load the first sheet
            df = xl.parse(sheet_names[0])
            crosswalk_data[file.stem] = df

            # Write top of dataframe to file
            out.write(df.head().to_string(index=False))
            out.write("\n" + "-"*60 + "\n")
        except Exception as e:
            out.write(f"Failed to load {file.name}: {e}\n")

print(f"Preview written to: {output_file}")

Preview written to: ..\data\processed\crosswalk_preview_output.txt


In [ ]:
# Attempt to concatenate all dataframes if they align
merged_df = pd.DataFrame()
for year, df in crosswalk_data.items():
    df = df.copy()
    df["Year"] = year  # Add a column for the year label
    merged_df = pd.concat([merged_df, df], ignore_index=True, sort=False)

# Drop duplicates based on first 3 columns
initial_row_count = len(merged_df)
merged_df.drop_duplicates(subset=merged_df.columns[:3].tolist(), inplace=True)
final_row_count = len(merged_df)
print(f"Dropped {initial_row_count - final_row_count} duplicate rows based on first 3 columns.")

# Save merged dataframe to processed directory
merged_output_file = Path("../data/processed/merged_crosswalks.csv")
merged_df.to_csv(merged_output_file, index=False)
print(f"Merged crosswalks saved to: {merged_output_file}")

Dropped 0 duplicate rows.


OSError: Cannot save file into a non-existent directory: 'data\processed'